In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO

In [ ]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
# Project structure
PROJECT_ROOT = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(PROJECT_ROOT, "data", "detection")

IMAGES_DIR = os.path.join(DATA_PATH, "images")
LABELS_DIR = os.path.join(DATA_PATH, "labels")

IMAGES_TRAIN_DIR = os.path.join(IMAGES_DIR, "train")
IMAGES_VAL_DIR = os.path.join(IMAGES_DIR, "val")

LABELS_TRAIN_DIR = os.path.join(LABELS_DIR, "train")
LABELS_VAL_DIR = os.path.join(LABELS_DIR, "val")

In [ ]:
for p in (IMAGES_TRAIN_DIR, IMAGES_VAL_DIR, LABELS_TRAIN_DIR, LABELS_VAL_DIR):
    if not os.path.exists(p):
        print("WARNING: path does not exist:", p)

In [ ]:
yaml_path = os.path.join(DATA_PATH, "face_detection.yaml")
with open(yaml_path, "w") as f:
    f.write(f"""
path: {DATA_PATH}
train: images/train
val: images/val
nc: 1
names: ['face']
""")
print("YAML file created at:", yaml_path)

In [ ]:
MODEL_PATH = os.path.join(PROJECT_ROOT, "models", "face_detection", "yolo11s.pt")
MODEL_RUNS_DIR = os.path.join(PROJECT_ROOT, "models", "face_detection", "runs")

In [ ]:
model = YOLO(MODEL_PATH)
print("Loaded model:", model)

In [ ]:
results = model.train(
    # DATA & MODEL
    data=yaml_path,
    model=MODEL_PATH,
    pretrained=True,

    # EPOCHS & IMAGE SIZE
    epochs=50,                 # allow proper convergence; early stop will cut it sooner if needed
    imgsz=640,                 # ↑ accuracy; if OOM -> 512 or 448
    batch=4,                   # keep small for low VRAM
    workers=2,                 # Windows-friendly

    # OPTIMIZATION
    optimizer="AdamW",
    lr0=7e-4,                  # slightly lower than 1e-3 for stability on small batches
    lrf=0.01,                  # final LR factor for cosine
    cos_lr=True,               # cosine LR schedule
    warmup_epochs=3,
    weight_decay=0.01,
    patience=20,               # early stop window (validation-based)
    amp=True,                  # mixed precision if GPU supports; silently falls back if not

    # AUGMENTATION (moderate, classroom-friendly)
    multi_scale=True,          # random scale each batch (good for small/varied faces)
    degrees=15.0,              # slight rotations for tilted heads
    shear=10.0,
    perspective=0.0005,
    translate=0.10,
    scale=0.50,                # allow zoom in/out
    fliplr=0.5,                # horizontal flips are fine; avoid vertical flips for faces
    flipud=0.0,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    mosaic=0.2,                # light mosaic (too much can distort faces)
    mixup=0.05,                # small; higher can hurt face boxes
    copy_paste=0.0,            # not great for faces

    # LOSS BALANCING (single class)
    box=7.5,                   # emphasize localization
    cls=0.2,                   # down-weight classification (only one class)
    dfl=1.5,

    # PROJECT STRUCTURE
    project=MODEL_RUNS_DIR,
    name="face_yolo11s",

    # STORAGE / HOUSEKEEPING
    save=True,                 # keep final weights
    save_period=1,            
    exist_ok=True,
    cache=False,
    plots=False,
    verbose=False,

    # DEVICE
    device=0                   # GPU 0; set "cpu" if necessary
)

In [ ]:
# RESUME TRAINING FROM CHECKPOINT
# Point to the 'last.pt' of your crashed run
resume_model_path = os.path.join(MODEL_RUNS_DIR , "face_yolo11s", "weights", "last.pt")

model = YOLO(resume_model_path)

# Resume training from where it stopped
results = model.train(
    resume=True,               # <--- this is the key
)

In [ ]:
metrics = model.val()  # runs validation and returns metrics dict
print("Validation metrics:", metrics)